# Potato Chips Retail Strategy and Analytics

## Part2 - Experimentation and uplift testing

Based on the recommendations from the findings on customer group & preferred brands of chips, three stores were chosen for  a trial period of **Feb 2019 to Apr 2019.**

The objective of this analysis is to assess the impact of the new trial layouts with a data driven recommendation to whether or not the trial layout should be rolled out to all other stores.

<b>The trial stores chosen are 77, 86 and 88.</b>

The workflow followed for this analysis can be summarized as below: 
- Define metrics to quantify the performance of the trial store as compared to other stores. 
- Identify the stores that show similarities in performance with the trial store using the defined metrics for a pre-trial period. 
- The most effective store would be the control store. 
- Scale the performance of the control store so as to compare it with the trial store for the trial period. 
- Assess the significance of percentage difference for each of the trial-control store combination. 
- Analyze performance of the trial stores as compared to the 5-95% confidence interval of the control store metrics.

The primary metrics used to match a trial store with a control store are: 
- Monthly overall sales revenue
- Monthly number of customers

## Import Libraries

In [1]:
import numpy as np
import pandas as pd
from datetime import datetime as dt

import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

import plotly.io as pio
pio.templates.default = "plotly_white"

from scipy import stats

pd.set_option('mode.chained_assignment', None)

## Read dataset

In [2]:
chips_retail = pd.read_csv("chips_retail.csv")

## Define metrics

Here we define the following metrics/ measures based on which we will compare the trial store with other stores. These metrics are calculated for all the stores and for all the months.  

- Total Sales for the month
- Average Sales for the month
- Distinct No. of Customers for the month
- Distinct No. of Transaction for the month
- Average chips purchased per transaction for the month

### Transform chips retail data from day-wise to month-wise

In [3]:
# create new column for grouping data month wise - format is yyyymm 
chips_retail["month_id"] = pd.to_datetime(chips_retail["date"]).dt.strftime("%Y%m") #.astype(int)

In [4]:
# calculate measures/ metrics overtime 
overtime_metrics = pd.pivot_table(data = chips_retail,
                                  index = ["month_id", "store_nbr"],
                                  values = ["lylty_card_nbr", "tot_sales", "txn_id", "prod_qty"],
                                  aggfunc = {"tot_sales" : [np.sum, np.mean], 
                                             "lylty_card_nbr" : pd.Series.nunique,
                                             "txn_id" : pd.Series.nunique,
                                             "prod_qty" : np.mean}
                                 ).reset_index()

# flatten the resultant multi-indexed column names of the dataframe
overtime_metrics.columns = ['_'.join(str(s).strip() for s in col if s) for col in overtime_metrics.columns]

# new column names to replace old column names 
new_cols = {"lylty_card_nbr_nunique": "n_customers", # no. of unique customers at the store 
            "prod_qty_mean" : "qty_per_txn",  # no. of chips purchased per transaction
            "tot_sales_mean" : "avg_sales",  # avg sales per unit qty for the month
            "tot_sales_sum" : "total_sales", # total sales for the month
            "txn_id_nunique" : "n_transactions"} # no. of unique transactions for the month

# rename column names of dataframe
overtime_metrics = overtime_metrics.rename(new_cols, axis = 1)

# dataframe shape
print(overtime_metrics.shape)

overtime_metrics.head(7)

(3169, 7)


month_id  store_nbr  n_customers  qty_per_txn  avg_sales  total_sales  \
0   201807          1           49     1.192308   3.978846        206.9   
1   201807          2           39     1.121951   3.678049        150.8   
2   201807          3          112     1.963768   8.736957       1205.7   
3   201807          4          128     1.987500   8.749375       1399.9   
4   201807          5           93     2.000000   6.766667        812.0   
5   201807          6           48     1.431373   5.098039        260.0   
6   201807          7          100     1.900000   8.539167       1024.7   

   n_transactions  
0              52  
1              41  
2             138  
3             158  
4             120  
5              51  
6             120

## Select stores which have data for all 12 months - Segregate data for pre-trial months

A non-trial store needs to fulfill two basic necessities to be a potential control store:
- Data should be present for all the 12 months as it is available for the trial store 
- Metrics performance should be very similar to the trial store for the pre-trial period i.e. from 2018-Jul to 2019-Jan. 

Here we have select only those potential control stores that have data for all the 12 months. 

After these stores are selected, we segregate data for the pre-trial periods. 

In [5]:
# stores that do not have full year data
exception_stores = overtime_metrics.groupby(["store_nbr"]).filter(lambda x: len(x) < 12)["store_nbr"].unique()

# pre-trial months data for the stores which have data for the full year
pre_trial_metrics = overtime_metrics[(overtime_metrics["month_id"].astype(int) < 201902) & 
                                     (overtime_metrics["store_nbr"].isin(exception_stores) != True)]

# sort the results by month_id
pre_trial_metrics = pre_trial_metrics.sort_values(by = "month_id").reset_index(drop = True)

pre_trial_metrics.head()

month_id  store_nbr  n_customers  qty_per_txn  avg_sales  total_sales  \
0   201807          1           49     1.192308   3.978846        206.9   
1   201807        173           49     2.000000   8.673077        451.0   
2   201807        174           39     1.925000   8.440000        337.6   
3   201807        175          100     1.888889   8.244444        964.6   
4   201807        176           51     1.415094   5.418868        287.2   

   n_transactions  
0              52  
1              52  
2              40  
3             115  
4              53

In [6]:
# from all the stores present select only non-trial stores
non_trial_stores = pre_trial_metrics[pre_trial_metrics["store_nbr"].isin([77, 86, 88]) == False]["store_nbr"].unique()

## Helpful functions to compare metrics between Trial and Non-Trial Stores

Few functions have been created to aid in calculating and comparing metrics between the trial and non-trial stores: 

- Function **calc_corr_coef** - to calculate the correlation coefficients between the metrics of each of the trial & non-trial store pair.

- Function **calc_distance** - to calculate the absolute magnitude distance between the metrics of each of the trial & non-trial store pair.

- Function **combine_calc** - to execute functions calc_corr_coef and calc_distance and combine their results as final output.

In [7]:
"""Calculate correlation coefficient between metrics"""
def calc_corr_coef(input_metrics, metric_col, trial_store):

    # empty dataframe to collect correlation coefficients
    output_df = pd.DataFrame()

    # year_month series
    year_month = input_metrics[input_metrics["store_nbr"] == trial_store]["month_id"].values
    
    # to generate the store number array  
    series_size = len(year_month) 
    
    # column name
    col_name = metric_col + "_corr_coef"

    # loop to run for all the non-trial stores
    for each_store in non_trial_stores: 

        # potential control store data
        x = input_metrics[input_metrics["store_nbr"] == each_store][metric_col]
        
        # trial store data
        y = input_metrics[input_metrics["store_nbr"] == trial_store][metric_col]

        # calculate correlation between metrics of each store with that of the trial store
        corr_coef = np.corrcoef(x,y, rowvar = False)[0][1]
        
        # dataframe to collect correlation coefficients for each store for each year_month
        corr_coef_df = pd.DataFrame([each_store, corr_coef]).T
        
        # concatenate corr_coef_df for each store to get one single output
        output_df = output_df.append(corr_coef_df, ignore_index = True)
    
    # rename columns of the output dataframe
    output_df.columns =  ["control_store", col_name]
    
    # append trial_store to distinguish
    output_df["trial_store"] = trial_store

    # return output dataframe containg correlation coefficients    
    return output_df

In [8]:
"""Calculate absolute magnitude distance between metrics"""
def calc_distance(input_metrics, metric_col, trial_store):
    
    # empty output dataframe
    output_df = pd.DataFrame()
    
    # year_month series from trial store data
    year_month = input_metrics.loc[input_metrics["store_nbr"] == trial_store, "month_id"].values
    
    # temp variable - column name 
    col_name = metric_col + "_scaled"

    # loop to run for all the non_trial stores
    for each_store in non_trial_stores: 
        
        # potential control store data
        x = input_metrics[input_metrics["store_nbr"] == each_store][metric_col].reset_index(drop = True)

        # trial store data
        y = input_metrics[input_metrics["store_nbr"] == trial_store][metric_col].reset_index(drop = True)

        # calculate magnitude of distance between values of each store and trial store
        distance = x.subtract(y).abs()
        
        # create data frame to combine magnitude distance for each store
        mag_dist = pd.DataFrame(data = [[each_store] * len(year_month), year_month, distance]).T
        
        # rename columns of the mag_dist dataframe
        mag_dist.columns = ["control_store","month_id","dist"]
    
        # concatenate result of each store to obtain a single output
        output_df = pd.concat([output_df, mag_dist])
    
    # calculate scaled_values for distance magnitude
    output_df[col_name] = output_df.groupby("control_store")["dist"].apply(
                               lambda dist: 1 - (dist - dist.min())/(dist.max() - dist.min()))
    
    # add trial store to the output dataframe
    output_df["trial_store"] = trial_store
    
    # some cleanup done on output dataframe 
    output_df = output_df[["control_store", "month_id", "trial_store", col_name]]
    
    # return the output as dataframe with store_number and scaled_distance
    return output_df

In [9]:
"""To obtain a combined output of calc_corr_coef and calc_distance functions"""
def combine_calc(input_metric, metric_col, trial_store):
    
    # execute correlation coefficient calculation function and assign to a dataframe
    corr_coef_df = calc_corr_coef(input_metric, metric_col, trial_store)
    
    # execute scaling calculation function and assign to a dataframe
    scaled_df = calc_distance(input_metric, metric_col, trial_store)
    
    # merge the two dataframes
    result_df = scaled_df.merge(corr_coef_df, left_on = ["control_store", "trial_store"],
                                              right_on = ["control_store", "trial_store"])
    
    return result_df

## Execute functions to calculate Correlation Coefficient and Magnitude Distance

In [10]:
# metrics columns to be used for comparing performance
metric_cols = ["total_sales", "n_customers"]

final_df = pd.DataFrame()

# loop to excute functions for all 3 trial stores
for trial_store in [77, 86, 88]: 
    df1 = combine_calc(pre_trial_metrics, metric_cols[0], trial_store)    
    df2 = combine_calc(pre_trial_metrics, metric_cols[1], trial_store)
    
    df = df1.merge(df2, left_on = ["control_store", "month_id", "trial_store"], 
                        right_on = ["control_store", "month_id", "trial_store"])
    
    final_df = pd.concat([final_df, df], ignore_index = True)

In [11]:
final_df.head(7).append(final_df.tail(7))

control_store month_id  trial_store total_sales_scaled  \
0                1   201807           77                  0   
1                1   201808           77           0.142857   
2                1   201809           77           0.493878   
3                1   201810           77                  1   
4                1   201811           77           0.506122   
5                1   201812           77           0.165986   
6                1   201901           77           0.548299   
5390            95   201807           88           0.878138   
5391            95   201808           88           0.608767   
5392            95   201809           88                  0   
5393            95   201810           88           0.714805   
5394            95   201811           88            0.40633   
5395            95   201812           88           0.716624   
5396            95   201901           88                  1   

      total_sales_corr_coef n_customers_scaled  n_customers_corr_coef  
0                  0.075218           0.882353               0.322168  
1                  0.075218           0.705882               0.322168  
2                  0.075218                  0               0.322168  
3                  0.075218           0.588235               0.322168  
4                  0.075218           0.705882               0.322168  
5                  0.075218           0.764706               0.322168  
6                  0.075218                  1               0.322168  
5390               0.119513            0.52381               0.244461  
5391               0.119513           0.238095               0.244461  
5392               0.119513                  0               0.244461  
5393               0.119513           0.619048               0.244461  
5394               0.119513           0.142857               0.244461  
5395               0.119513           0.619048               0.244461  
5396               0.119513                  1               0.244461

## Calculate score to rank non-trial stores

Now that we have calculated correlation coefficients & magnitude distance, we need to create a composite score based on both these values, to rank the non-trial stores. 

The final score to rank the non-trial stores has been calculated as follows: 
- calculate mean of correlation coefficients & magnitude distance for sales - sales score
- calculate mean of correlation coefficients & magnitude distance for customer count - customer score
- calculate mean of the sales & customer score

This calculation gives equal weightage to each of the correlation coefficient and metric magnitude distance.

<font color = "gray">*(I tried and experimented with maximum rank (irrespective of the month) and mean rank (for all the months). I was able to understand the mean rank better so I have used that for this analysis.)*</font>

In [12]:
final_df["sales_score"] = ((final_df["total_sales_scaled"] + final_df["total_sales_corr_coef"])/2).astype(float)
final_df["cust_score"] = ((final_df["n_customers_scaled"] + final_df["n_customers_corr_coef"])/2).astype(float)
final_df["final_score"] = ((final_df["sales_score"] + final_df["cust_score"])/2).astype(float)

final_df.head()

control_store month_id  trial_store total_sales_scaled  \
0             1   201807           77                  0   
1             1   201808           77           0.142857   
2             1   201809           77           0.493878   
3             1   201810           77                  1   
4             1   201811           77           0.506122   

   total_sales_corr_coef n_customers_scaled  n_customers_corr_coef  \
0               0.075218           0.882353               0.322168   
1               0.075218           0.705882               0.322168   
2               0.075218                  0               0.322168   
3               0.075218           0.588235               0.322168   
4               0.075218           0.705882               0.322168   

   sales_score  cust_score  final_score  
0     0.037609    0.602261     0.319935  
1     0.109037    0.514025     0.311531  
2     0.284548    0.161084     0.222816  
3     0.537609    0.455202     0.496405  
4     0.290670    0.514025     0.402348

## Identify & Visualize top 5 stores with highest mean Final Score

Based on the mean final score, let's identify the top 5 stores with highest rank score values.

In [13]:
for trial_store in [77, 86, 88]:
    temp_df = final_df[final_df["trial_store"] == trial_store].groupby("control_store",as_index = False)["final_score"].mean()
    temp_df = temp_df.sort_values("final_score", ascending = False).head()

    print("Top 5 Potential Control Stores for trial store {}".format(trial_store))
    print("-------------------------------------------------")
    print(temp_df.reset_index(drop = True), "\n")

Top 5 Potential Control Stores for trial store 77
-------------------------------------------------
   control_store  final_score
0            233     0.775294
1            254     0.695106
2            119     0.688938
3            167     0.674464
4             41     0.672886 

Top 5 Potential Control Stores for trial store 86
-------------------------------------------------
   control_store  final_score
0            260     0.708732
1            155     0.704381
2            109     0.677802
3            176     0.648407
4            114     0.642607 

Top 5 Potential Control Stores for trial store 88
-------------------------------------------------
   control_store  final_score
0            178     0.705994
1             14     0.698455
2            113     0.696395
3            253     0.605338
4            237     0.584667 



In [14]:
print("Use/click the legend to select two or more stores to compare")

for trial_store in [77, 86, 88]:
    
    # calculate mean final score for all the stores
    temp_df = (final_df[final_df["trial_store"] == trial_store].groupby("control_store",
                                                                        as_index = False)["final_score"].mean())
    # sort and select top 5 potential control stores with maximum final score mean
    temp_df = temp_df.sort_values("final_score", ascending = False).head()
    
    # create a temporary df for each of the trial store comparison with it's potential control stores
    df = (pre_trial_metrics[pre_trial_metrics["store_nbr"].isin(list(temp_df["control_store"])+[trial_store])])
    
    # set the month_id column format as YearMonth
    df["month_id"] = pd.to_datetime(df["month_id"], format = "%Y%m").dt.strftime("%Y%b")    
    
    # common settings
    
    # colors for each store line
    colors = px.colors.diverging.Portland + ["rgb(34,139,34)"]
    
    # store label
    yaxis_labels = ["Store"+ str(each) for each in df["store_nbr"].unique()]
    
    # month_id as x co-ordinates
    x_cords = [list(df["month_id"].unique())] * len(yaxis_labels)
    
    y_cords_sales = [] 
    y_cords_cust = []

    # loop to execute for each_store
    for each_store in df["store_nbr"].unique():
        
        # total sales for each store
        y = df[df["store_nbr"]== each_store]["total_sales"].values
        # append to y-sales co-ordinates list of lists
        y_cords_sales.append(list(y.round(2)))
        
        # n_customers for each store
        y = df[df["store_nbr"]== each_store]["n_customers"].values
        # append to y-customer co-ordinates list of lists        
        y_cords_cust.append(list(y.round(2)))

    # subplot instantiation    
    fig = make_subplots(rows = 1, cols = 2, subplot_titles = ["Total Sales", "Customer Walkins"]) 

    # add 6 traces by loop (Top 5 ranked stores + trial store)
    for i in range(0, 6):
        
        fig.add_trace(go.Scatter(x = x_cords[i], y = y_cords_sales[i], 
                                 line = {"color" : colors[i]},
                                 name = yaxis_labels[i], showlegend = True, 
                                 legendgroup = yaxis_labels[i]), row = 1, col = 1)
        
        fig.add_trace(go.Scatter(x = x_cords[i], y = y_cords_cust[i], 
                                 line = {"color" : colors[i]},
                                 name = yaxis_labels[i], showlegend = False, 
                                 legendgroup = yaxis_labels[i]), row = 1, col = 2)
        
        fig.update_xaxes(showline = True, showgrid = False, linewidth = 3)
        fig.update_yaxes(showline = True, showgrid = True, linewidth = 3)
    
    fig.update_layout(title = {"text": "Comparison of Sales and Customers Counts with Trial Store {}".format(trial_store), 
                               "font_size": 20, "x" : 0.5})
        
    fig.show()

Use/click the legend to select two or more stores to compare


### Observations: 

- <u>Trial Store 77</u>
    - In the case of trial store 77 the store 233, scores highest among the top 5 potential control stores. 
    - Similar result is visible from the bar plot. 
    - For trial store 77, the most applicable control store is 233
    

- <u>Trial Store 86</u>
    - In the case of trial store 86 the store 155, scores second highest among the top 5 potential control stores. 
    - However, based on the plot, it is evident that 155 resembles a lot more to the trial store 86 in the pre-trial performance, barring fluctuations for a couple of months.
    - For trial store 86, the most applicable control store is 155.
    
    
- <u>Trial Store 88 (This one is tricky!)</u>
    - In the case of trial store 88 the store 237, scores last among the top 5 potential control stores. However, based on the plot, store 237 is very close to trial store 88 for both the metrics.
    - Since the correlation coefficient between store 88 and store 237 is 0.31 for Sales, it brings down the final score as compared to the other stores. The correlation coefficient for customer visits however is highest at 0.95. (A comaprison plot is below.)
    
    - For trial store 88, the most applicable control store is 237. 

In [15]:
select_cols = ["control_store", "total_sales_corr_coef", "n_customers_corr_coef"]

df = final_df[(final_df["trial_store"]== 88) & (final_df["control_store"].isin([178, 113, 14, 253, 237]))][select_cols]
df = df.drop_duplicates().reset_index(drop = True)

x_cord = ["Store" + str(each) for each in df["control_store"]]

fig = go.Figure()
fig.add_trace(go.Bar(x = x_cord, y = df["total_sales_corr_coef"],
                     text = df["total_sales_corr_coef"].round(3), textposition = "outside",
                     name = "Sales Coef", marker_color = px.colors.diverging.Portland[0]))

fig.add_trace(go.Bar(x = x_cord, y = df["n_customers_corr_coef"], 
                     text = df["n_customers_corr_coef"].round(3), textposition = "outside",
                     name = "Customer Coef", marker_color = px.colors.diverging.Portland[1]))

fig.update_layout(height = 400, title = "Correlation Coefficients For Potential Control Stores vs Trial Store 88",
                  #xaxis = {"tickmode" : "array", "tickvals" : [14, 113, 178, 237, 253]}
                  yaxis = {"range" : [0, 1.15]}, bargap = 0.5, font_size = 15,
                  xaxis = {"showline" : True, "linewidth" : 1, "linecolor" : "rgb(67, 67, 67)"}
                 )
fig.show()

## Effect of trial period on trial stores performance for Chips Sales

In order to understand the effect of trial period on trial stores, we would need to bring the metrics data of the control stores to a comparable level to that of trial store. 

To achieve this, the workflow applied is as follows: 
- Calculate a scaling factor using the pre-trial period data for each of the metrics for each trial store
- Scale the values of respective control store by applying the scaling factor
- Calculate the absolute percentage difference between trial store and the control store
- Assess if the percentage difference is significant i.e. if trial stores values falls within the critical values of the control store or outside

### Scale the control store sales to comparable values with that of the  trial store

In [16]:
# trial control pair
trial_ctrl_pair = [(77, 233), (86, 155), (88, 237)]

# select 12 months data 
trial_control_df = overtime_metrics[overtime_metrics["store_nbr"].isin([77, 233, 86, 155, 88, 237])]

# metrics to be scaled
metric_cols = ["total_sales", "n_customers"]

for each_col in metric_cols:
    for each_pair in trial_ctrl_pair:
        scaling_factor = (pre_trial_metrics[pre_trial_metrics["store_nbr"] == each_pair[0]][each_col].sum()/ (
            pre_trial_metrics[pre_trial_metrics["store_nbr"] == each_pair[1]][each_col].sum()))

        metric_col = trial_control_df[trial_control_df["store_nbr"] == each_pair[1]][each_col]

        trial_control_df.loc[trial_control_df["store_nbr"] == each_pair[1], each_col] = metric_col * scaling_factor
        

In [17]:
trial_control_df[:10]

month_id  store_nbr  n_customers  qty_per_txn  avg_sales  total_sales  \
73    201807         77    51.000000     1.527273   5.396364   296.800000   
82    201807         86    99.000000     1.992063   7.080952   892.200000   
84    201807         88   129.000000     2.000000   8.562092  1310.000000   
150   201807        155   101.000000     2.000000   7.396800   896.922236   
227   201807        233    51.171141     1.629630   5.383333   297.565550   
231   201807        237   127.276836     2.000000   8.940741  1450.657086   
338   201808         77    47.000000     1.541667   5.322917   255.500000   
346   201808         86    94.000000     1.919643   6.821875   764.050000   
348   201808         88   131.000000     1.893750   8.273750  1323.800000   
414   201808        155    91.000000     1.908333   6.522500   759.269991   

     n_transactions  
73               55  
82              126  
84              153  
150             123  
227              54  
231             162  
338              48  
346             110  
348             158  
414             119

### Calculate absolute percentage difference between control store and trial store

In [18]:
# segregate and pivot resultant dataframe

sales_pv = trial_control_df.pivot(index = "month_id", columns = "store_nbr", 
                                          values = "total_sales").reset_index()

customers_pv = trial_control_df.pivot(index = "month_id", columns = "store_nbr", 
                                          values = "n_customers").reset_index()

In [19]:
for each_pair in trial_ctrl_pair:
    
    # create column for each of the trial and control store pair
    col_name = "_".join(("pct_diff", str(each_pair[0]), str(each_pair[1])))
    
    # calculate percentage difference for sales
    sales_pv[col_name] = abs((sales_pv[each_pair[1]] - sales_pv[each_pair[0]])/(
                             (sales_pv[each_pair[0]] + sales_pv[each_pair[1]])/ 2))
    
    # calculate percentage difference for n_customers
    customers_pv[col_name] = abs((customers_pv[each_pair[1]] - customers_pv[each_pair[0]])/(
                              (customers_pv[each_pair[0]] + customers_pv[each_pair[1]])/ 2))

In [20]:
sales_pv

store_nbr month_id     77       86       88         155         233  \
0           201807  296.8   892.20  1310.00  896.922236  297.565550   
1           201808  255.5   764.05  1323.80  759.269991  292.652187   
2           201809  225.2   914.60  1423.00  984.034086  233.998916   
3           201810  204.5   948.40  1352.40  934.948790  190.085733   
4           201811  245.3   918.00  1382.80  871.894555  216.597421   
5           201812  267.3   841.20  1325.20  824.361363  286.408121   
6           201901  204.4   841.40  1266.40  848.418979  181.692071   
7           201902  235.0   913.20  1370.20  864.522060  249.762622   
8           201903  278.5  1026.80  1477.20  780.320405  203.802205   
9           201904  263.5   848.20  1439.40  819.317024  162.345704   
10          201905  299.3   889.30  1308.25  895.224622  352.533799   
11          201906  264.7   838.00  1354.60  831.539845  226.219424   

store_nbr          237  pct_diff_77_233  pct_diff_86_155  pct_diff_88_237  
0          1450.657086         0.002576         0.005279         0.101901  
1          1369.931485         0.135554         0.006276         0.034251  
2          1324.260425         0.038323         0.073141         0.071882  
3          1350.401097         0.073060         0.014284         0.001479  
4          1399.777923         0.124281         0.051517         0.012203  
5          1266.971288         0.069019         0.020220         0.044927  
6          1221.600696         0.117630         0.008307         0.036012  
7          1406.989143         0.060907         0.054764         0.026494  
8          1210.082775         0.309755         0.272787         0.198801  
9          1206.477165         0.475075         0.034642         0.176065  
10         1201.168906         0.163335         0.006640         0.085343  
11         1155.397690         0.156769         0.007739         0.158727

In [21]:
customers_pv

store_nbr month_id    77     86     88    155        233         237  \
0           201807  51.0   99.0  129.0  101.0  51.171141  127.276836   
1           201808  47.0   94.0  131.0   91.0  48.161074  134.237288   
2           201809  42.0  103.0  124.0  103.0  42.140940  125.288136   
3           201810  37.0  109.0  123.0  108.0  35.117450  122.305085   
4           201811  41.0  100.0  130.0  101.0  40.134228  131.254237   
5           201812  46.0   98.0  126.0   97.0  47.157718  123.299435   
6           201901  35.0   94.0  117.0   96.0  35.117450  116.338983   
7           201902  45.0  107.0  124.0   95.0  45.151007  125.288136   
8           201903  50.0  115.0  134.0   94.0  40.134228  118.327684   
9           201904  47.0  105.0  128.0   99.0  30.100671  119.322034   
10          201905  55.0  104.0  128.0  106.0  57.191275  128.271186   
11          201906  41.0   98.0  121.0   95.0  41.137584  118.327684   

store_nbr  pct_diff_77_233  pct_diff_86_155  pct_diff_88_237  
0                 0.003350         0.020000         0.013448  
1                 0.024402         0.032432         0.024411  
2                 0.003350         0.000000         0.010335  
3                 0.052208         0.009217         0.005666  
4                 0.021342         0.009950         0.009602  
5                 0.024855         0.010256         0.021665  
6                 0.003350         0.021053         0.005666  
7                 0.003350         0.118812         0.010335  
8                 0.218913         0.200957         0.124222  
9                 0.438370         0.058824         0.070175  
10                0.039063         0.019048         0.002116  
11                0.003350         0.031088         0.022332

### Assess the significance of percentage difference

Here our null hypothesis is that the trial period performance for trial store is same as the non-trial period.

$H_0: \bar {x} = \mu$ - Trial layouts have no significant impact on the the trial store during the trial periods. 

$H_\alpha : \bar {x} \neq \mu$ - Trial layouts impacted performance of the trial store, during trial period. 

$\alpha = 0.05 \space or \space 5\%$ - Level of significance.

<font color = "grey">(I don't fully comprehend this section:  
- Why degrees of freedom is 7 and not 6. The non-trial periods are from 2018-Jun to 2019-Jan making 7 months of pre-trial.</font>

If we are able to support that the t-score (Student's T-distribution) for percentage difference between trial store and control store is higher than the critical values at 5% and 95% i.e. outside the confidence interval of the control store, we can reject the null hypothesis that there was no significant impact on the sales of trial stores for the trial period.

At 5% significance we will be able to conclude that the trial layouts were successful and positively impacted the sales at the trial store. 

In [22]:
sales_std_dev = {}
ncust_std_dev = {}

for each_pair in trial_ctrl_pair:
    col_name = "pct_diff_" + str(each_pair[0]) + "_" + str(each_pair[1])
    sales_std_dev[each_pair] = sales_pv[sales_pv["month_id"] < "201902"][col_name].std()
    ncust_std_dev[each_pair] = customers_pv[customers_pv["month_id"] < "201902"][col_name].std()
    
print("Sales std dev for each trial-control pair")
print(sales_std_dev, "\n")

print("Customers count std dev for each trial-control pair")
print(ncust_std_dev)

Sales std dev for each trial-control pair
{(77, 233): 0.04892726069143223, (86, 155): 0.026376343626736243, (88, 237): 0.03441089754539568} 

Customers count std dev for each trial-control pair
{(77, 233): 0.01782540740663612, (86, 155): 0.010577873853063899, (88, 237): 0.007435464986617887}


In [23]:
output_df = pd.DataFrame()

for each_pair in trial_ctrl_pair:
    df = pd.DataFrame()
    
    col_name = "pct_diff_" + str(each_pair[0]) + "_" + str(each_pair[1])
    
    # t_values
    t_values = sales_pv[col_name] / sales_std_dev[each_pair]

    # 95% quantile
    qtl_95 = stats.t.ppf(q = 0.95, df = 6)
    
    df["month_id"] = sales_pv[sales_pv["month_id"].between("201902", "201904")]["month_id"]
    df["trial_control_combo"] = [each_pair] * 3
    df["metric"] = "total sales"
    df["pct_diff"] = sales_pv[col_name]
    df["t_values"] = t_values
    df["critical_value_at_0.95"] = qtl_95
    
    # append result for each trial-control combo
    output_df = pd.concat([output_df,df], ignore_index = True)
    
output_df

month_id trial_control_combo       metric  pct_diff   t_values  \
0   201902           (77, 233)  total sales  0.060907   1.244840   
1   201903           (77, 233)  total sales  0.309755   6.330932   
2   201904           (77, 233)  total sales  0.475075   9.709819   
3   201902           (86, 155)  total sales  0.054764   2.076270   
4   201903           (86, 155)  total sales  0.272787  10.342113   
5   201904           (86, 155)  total sales  0.034642   1.313370   
6   201902           (88, 237)  total sales  0.026494   0.769925   
7   201903           (88, 237)  total sales  0.198801   5.777268   
8   201904           (88, 237)  total sales  0.176065   5.116540   

   critical_value_at_0.95  
0                 1.94318  
1                 1.94318  
2                 1.94318  
3                 1.94318  
4                 1.94318  
5                 1.94318  
6                 1.94318  
7                 1.94318  
8                 1.94318

If we compare the t-scores with the 0.95 quantile with 6 degrees of freedom of the t-distribution, we can observe: 
- Sales have increased for Mar 2019 and Apr 2019 for Trial stores 77 and 88 
- For trial store 86, a huge difference in sales has been achieved for the month of Mar 2019 only.

In [24]:
## same calculation for n_customers
output_df = pd.DataFrame()

for each_pair in trial_ctrl_pair:
    df = pd.DataFrame()
    
    col_name = "pct_diff_" + str(each_pair[0]) + "_" + str(each_pair[1])
   
    # t_values
    t_values = customers_pv[col_name] / ncust_std_dev[each_pair]

    # 95% quantile
    qtl_95 = stats.t.ppf(q = 0.95, df = 6)
    
    df["month_id"] = customers_pv[customers_pv["month_id"].between("201902", "201904")]["month_id"]
    df["trial_control_combo"] = [each_pair] * 3
    df["metric"] = "n_customers"
    df["pct_diff"] = customers_pv[col_name]
    df["t_values"] = t_values
    df["critical_value_at_0.95"] = qtl_95
    
    output_df = pd.concat([output_df,df], ignore_index = True)

output_df

month_id trial_control_combo       metric  pct_diff   t_values  \
0   201902           (77, 233)  n_customers  0.003350   0.187939   
1   201903           (77, 233)  n_customers  0.218913  12.280947   
2   201904           (77, 233)  n_customers  0.438370  24.592452   
3   201902           (86, 155)  n_customers  0.118812  11.232114   
4   201903           (86, 155)  n_customers  0.200957  18.997857   
5   201904           (86, 155)  n_customers  0.058824   5.560997   
6   201902           (88, 237)  n_customers  0.010335   1.389894   
7   201903           (88, 237)  n_customers  0.124222  16.706680   
8   201904           (88, 237)  n_customers  0.070175   9.437935   

   critical_value_at_0.95  
0                 1.94318  
1                 1.94318  
2                 1.94318  
3                 1.94318  
4                 1.94318  
5                 1.94318  
6                 1.94318  
7                 1.94318  
8                 1.94318

If we compare the t-scores with the 0.95 quantile with 6 degrees of freedom of the t-distribution, we can observe: 
- Number of customes at the store have increased for Mar 2019 and Apr 2019 for Trial stores 77 and 88. 
- For trial store 86, all the three trial periods show significant increase in the no. of customer walkins at the store.

In [25]:
for each_pair in trial_ctrl_pair:
    int_col1 = str(each_pair[1]) + "_0.95_Interval"
    int_col2 = str(each_pair[1]) + "_0.05_Interval"
    
    sales_pv[int_col1] = sales_pv[each_pair[1]] * (1 + sales_std_dev[each_pair] * 2)
    sales_pv[int_col2] = sales_pv[each_pair[1]] * (1 - sales_std_dev[each_pair] * 2)
    
    customers_pv[int_col1] = customers_pv[each_pair[1]] * (1 + ncust_std_dev[each_pair] * 2)
    customers_pv[int_col2] = customers_pv[each_pair[1]] * (1 - ncust_std_dev[each_pair] * 2)

In [26]:
print("Use/click the legend to select two or more stores to compare")

for each_pair in trial_ctrl_pair:
    
    control = each_pair[1]
    trial = each_pair[0]
    
    int_col1 = str(control) + "_0.95_Interval"
    int_col2 = str(control) + "_0.05_Interval"
    
    sales_df = (sales_pv[(sales_pv["month_id"].between("201901", "201905"))]
                [["month_id", int_col2, control, trial, int_col1]])
    
    customers_df = (customers_pv[(customers_pv["month_id"].between("201901", "201905"))]
                    [["month_id", int_col2, control, trial, int_col1]])
    
    labels = ["Control 5% Interval", "Control " + str(control), "Trial " + str(trial), "Control 95% Interval"]
    
    line_colors = px.colors.diverging.Portland
    
    fig = make_subplots(rows = 1, cols = 2, subplot_titles = ["Total Sales", "Customer Walkins"])

    for i in range(1, 5):
        fig.add_trace(go.Scatter(x = pd.to_datetime(sales_df["month_id"], format = "%Y%m").dt.strftime("%Y%b"), 
                                 y = sales_df[sales_df.columns[i]],  
                                 mode = "markers+lines", marker = {"size" : 10},
                                 line = {"dash": "dot", "color": line_colors[i]},
                                 name = labels[i - 1], 
                                 legendgroup = labels[i - 1], 
                                 showlegend = True), row = 1, col = 1)
        
        fig.add_trace(go.Scatter(x = pd.to_datetime(sales_df["month_id"], format = "%Y%m").dt.strftime("%Y%b"), 
                                 y = customers_df[customers_df.columns[i]], 
                                 mode = "markers+lines", marker = {"size" : 10},
                                 line = {"dash": "dot", "color": line_colors[i]},
                                 name = labels[i - 1], 
                                 legendgroup = labels[i - 1], 
                                 showlegend = False), row = 1, col = 2)  
        
        fig.update_xaxes(showline = True, showgrid = False, linewidth = 3)
        fig.update_yaxes(showline = True, showgrid = True, linewidth = 3)
    
    fig.update_layout(height = 430, 
                      title = {"text": "Assessment of Performance of Trial Store {}".format(trial) +
                                        " with Control Store {}".format(control), 
                               "font_size" : 20})       
            
    fig.show()

Use/click the legend to select two or more stores to compare


## Observations & Conclusions:

By comparing the metrics - sales and no. of customers visited, of the trial store with the 
- scaled control stores and 
- the interval between 5% and 95% confidence of scaled control stores

we can conclude the following:

<u>For Trial Store 77:</u>

- The trial period has shown significant increase in sales for the months of Mar 2019 and Apr 2019. Although the sales have not increased much for Feb 2019 month. 
- The no. of customers who visited the stores for the trial period also show a similar trend to sales.
- We can say that the trial period for trial store 77 has **been successful for 2 out of 3 months.** 

<u>For Trial Store 86:</u>

- We can observe an increase of no. of customers who have visited the trial store 86 for all the three months.
- However, total sales do not resemble the same trend. Sales for Feb 2019 barely crosses the 95% confidence interval of the control store 155. 
- Total Sales for Mar 2019 is higher, but it again drops for Apr 2019 and is within the 5% and 95% confidence interval of the total sales of the control store 155. 
- For trial store 86, we would need to **look at other factors** that may be responsible for increase in the number of customers at the store. But, increase in sales for only **1 month out of the 3 trial months.** 

<u>For Trial Store 88:</u>

- Similar to trial store 77, we can see increase in sales for the months of Mar 2019 and Apr 2019 for trial store 88. 
- Neither the sales nor the no. of customers show any increase for the month of Feb 2019.
- For trial store 88, the trial period shows **favourable results for 2 out of the 3 months.**

<b> Overall, we can conclude that the trial period was successfull in increasing the sales and customer visits to the trial store. We can make significant impact by increasing the visibility of the favourable products to the respective target segment in other stores as well. </b>